In [1]:
import echopype as ep
import xarray as xr
import numpy as np
import pandas as pd

import echopype as ep
from echopype.utils.compute import _lin2log, _log2lin
from echopype.clean.utils import calc_transient_noise_pooled_Sv

In [2]:
func = "nanmedian"
transient_noise_threshold = 12.0
chunk=False
depth_bin="2m"
num_side_pings=2
exclude_above = 250

# Open raw, calibrate, and add depth
ed = ep.open_raw(
    "echopype/test_data/ek60/from_echopy/JR161-D20061118-T010645.raw",
    sonar_model="EK60"
)
ds_Sv = ep.calibrate.compute_Sv(ed)
ds_Sv = ep.consolidate.add_depth(ds_Sv)

# Select Sv subset
ds_Sv = ds_Sv.isel(ping_time=slice(294, 300), range_sample=slice(1770,1800))

if chunk:
    # Chunk Sv
    ds_Sv = ds_Sv.chunk("auto")

In [3]:
ep.clean.mask_transient_noise(
    ds_Sv.compute(),
    func="nanmedian",
)

<xarray.DataArray 'Sv' (channel: 3, ping_time: 6, range_sample: 30)> Size: 540B
array([[[False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
...
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False]]])
Coordinates:
  * channel       (channel) <U34 408B 'GPT  38 kHz 009072033fa5 1 ES38' ... '...
  * ping_time     (ping_time) datetime64[ns] 48B 2006-11-18T01:18:59.874197 ....
  * range_sample  (range_sample) int64 240B 1770 1771 1772 ... 1797 1798 1799

In [5]:
pooled_Sv = calc_transient_noise_pooled_Sv(ds_Sv, func, 2, num_side_pings, exclude_above)

In [11]:
pooled_Sv - ds_Sv["Sv"].compute()

<xarray.DataArray 'Sv' (channel: 3, ping_time: 6, range_sample: 30)> Size: 4kB
array([[[            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan],
        [            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan],
...
        [            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,  3.60418420e-01,
         -3.32740390e-01, -6.41427519e-01,  1.27434953e-01,
          2.86338938e-01,  3.84015760e-01,  3.50716096e-01,
         -7.32654668e-01,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan],
        [            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan]]])
Coordinates:
  * channel       (channel) <U34 408B 'GPT  38 kHz 009072033fa5 1 ES38' ... '...
  * ping_time     (ping_time) datetime64[ns] 48B 2006-11-18T01:18:59.874197 ....
  * range_sample  (range_sample) int64 240B 1770 1771 1772 ... 1797 1798 1799

In [7]:
# Compute min and max values
depth_values_min = ds_Sv["depth"].min()
depth_values_max = ds_Sv["depth"].max()
ping_time_index_min = 0
ping_time_index_max = len(ds_Sv["ping_time"])

# Create ping time indices array
ping_time_indices = xr.DataArray(
    np.arange(len(ds_Sv["ping_time"]), dtype=int),
    dims=["ping_time"],
    coords=[ds_Sv["ping_time"]],
    name="ping_time_indices",
)

# Check appropriate NaN boundaries
within_mask = (
    (ds_Sv["depth"] - depth_bin >= depth_values_min) &
    (ds_Sv["depth"] + depth_bin <= depth_values_max) &
    (ds_Sv["depth"] - depth_bin >= exclude_above) &
    (ping_time_indices - num_side_pings >= ping_time_index_min) &
    (ping_time_indices + num_side_pings <= ping_time_index_max)
)
unique_pool_boundary_values = np.unique(
    pooled_array.where(~within_mask)
)

# Check that NaN is the only pool boundary unique value
assert np.isclose(unique_pool_boundary_values, np.array([np.nan]), equal_nan=True)